In [57]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.metrics import classification_report, precision_score, recall_score, accuracy_score, f1_score
from sklearn.metrics import plot_confusion_matrix, confusion_matrix 
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.preprocessing import StandardScaler, OneHotEncoder, FunctionTransformer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.linear_model import LogisticRegression


from xgboost import XGBClassifier

from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImPipeline

import warnings
warnings.filterwarnings('ignore') 

In [29]:
df = pd.read_csv('data/telco_churn_data.csv')

df.head()

,state,account length,area code,phone number,international plan,voice mail plan,number vmail messages,total day minutes,total day calls,total day charge,...,total eve calls,total eve charge,total night minutes,total night calls,total night charge,total intl minutes,total intl calls,total intl charge,customer service calls,churn
0,KS,128,415,382-4657,no,yes,25,265.1,110,45.07,...,99,16.78,244.7,91,11.01,10.0,3,2.70,1,False
1,OH,107,415,371-7191,no,yes,26,161.6,123,27.47,...,103,16.62,254.4,103,11.45,13.7,3,3.70,1,False
2,NJ,137,415,358-1921,no,no,0,243.4,114,41.38,...,110,10.30,162.6,104,7.32,12.2,5,3.29,0,False
3,OH,84,408,375-9999,yes,no,0,299.4,71,50.90,...,88,5.26,196.9,89,8.86,6.6,7,1.78,2,False
4,OK,75,415,330-6626,yes,no,0,166.7,113,28.34,...,122,12.61,186.9,121,8.41,10.1,3,2.73,3,False


In [30]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3333 entries, 0 to 3332
Data columns (total 21 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   state                   3333 non-null   object 
 1   account length          3333 non-null   int64  
 2   area code               3333 non-null   int64  
 3   phone number            3333 non-null   object 
 4   international plan      3333 non-null   object 
 5   voice mail plan         3333 non-null   object 
 6   number vmail messages   3333 non-null   int64  
 7   total day minutes       3333 non-null   float64
 8   total day calls         3333 non-null   int64  
 9   total day charge        3333 non-null   float64
 10  total eve minutes       3333 non-null   float64
 11  total eve calls         3333 non-null   int64  
 12  total eve charge        3333 non-null   float64
 13  total night minutes     3333 non-null   float64
 14  total night calls       3333 non-null   

In [31]:
df['churn'] = df['churn'].astype('int')

In [32]:
df.describe()

,account length,area code,number vmail messages,total day minutes,total day calls,total day charge,total eve minutes,total eve calls,total eve charge,total night minutes,total night calls,total night charge,total intl minutes,total intl calls,total intl charge,customer service calls,churn
count,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000
mean,101.064806,437.182418,8.099010,179.775098,100.435644,30.562307,200.980348,100.114311,17.083540,200.872037,100.107711,9.039325,10.237294,4.479448,2.764581,1.562856,0.144914
std,39.822106,42.371290,13.688365,54.467389,20.069084,9.259435,50.713844,19.922625,4.310668,50.573847,19.568609,2.275873,2.791840,2.461214,0.753773,1.315491,0.352067
min,1.000000,408.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,23.200000,33.000000,1.040000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,74.000000,408.000000,0.000000,143.700000,87.000000,24.430000,166.600000,87.000000,14.160000,167.000000,87.000000,7.520000,8.500000,3.000000,2.300000,1.000000,0.000000
50%,101.000000,415.000000,0.000000,179.400000,101.000000,30.500000,201.400000,100.000000,17.120000,201.200000,100.000000,9.050000,10.300000,4.000000,2.780000,1.000000,0.000000
75%,127.000000,510.000000,20.000000,216.400000,114.000000,36.790000,235.300000,114.000000,20.000000,235.300000,113.000000,10.590000,12.100000,6.000000,3.270000,2.000000,0.000000
max,243.000000,510.000000,51.000000,350.800000,165.000000,59.640000,363.700000,170.000000,30.910000,395.000000,175.000000,17.770000,20.000000,20.000000,5.400000,9.000000,1.000000


In [33]:
df.isna().sum()

state                     0
account length            0
area code                 0
phone number              0
international plan        0
voice mail plan           0
number vmail messages     0
total day minutes         0
total day calls           0
total day charge          0
total eve minutes         0
total eve calls           0
total eve charge          0
total night minutes       0
total night calls         0
total night charge        0
total intl minutes        0
total intl calls          0
total intl charge         0
customer service calls    0
churn                     0
dtype: int64

In [34]:
df['area code'] = df['area code'].astype('str')

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3333 entries, 0 to 3332
Data columns (total 21 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   state                   3333 non-null   object 
 1   account length          3333 non-null   int64  
 2   area code               3333 non-null   object 
 3   phone number            3333 non-null   object 
 4   international plan      3333 non-null   object 
 5   voice mail plan         3333 non-null   object 
 6   number vmail messages   3333 non-null   int64  
 7   total day minutes       3333 non-null   float64
 8   total day calls         3333 non-null   int64  
 9   total day charge        3333 non-null   float64
 10  total eve minutes       3333 non-null   float64
 11  total eve calls         3333 non-null   int64  
 12  total eve charge        3333 non-null   float64
 13  total night minutes     3333 non-null   float64
 14  total night calls       3333 non-null   

In [35]:
df.churn.value_counts()

0    2850
1     483
Name: churn, dtype: int64

In [36]:
df_obj_cols = df.select_dtypes(include='object')

for col in df_obj_cols:
    print(f'{col.title()} has the following different values: {df[col].unique()}')
    print(f'There are {df[col].nunique()} options for the {col.title()} feature')
    

State has the following different values: ['KS' 'OH' 'NJ' 'OK' 'AL' 'MA' 'MO' 'LA' 'WV' 'IN' 'RI' 'IA' 'MT' 'NY'
 'ID' 'VT' 'VA' 'TX' 'FL' 'CO' 'AZ' 'SC' 'NE' 'WY' 'HI' 'IL' 'NH' 'GA'
 'AK' 'MD' 'AR' 'WI' 'OR' 'MI' 'DE' 'UT' 'CA' 'MN' 'SD' 'NC' 'WA' 'NM'
 'NV' 'DC' 'KY' 'ME' 'MS' 'TN' 'PA' 'CT' 'ND']
There are 51 options for the State feature
Area Code has the following different values: ['415' '408' '510']
There are 3 options for the Area Code feature
Phone Number has the following different values: ['382-4657' '371-7191' '358-1921' ... '328-8230' '364-6381' '400-4344']
There are 3333 options for the Phone Number feature
International Plan has the following different values: ['no' 'yes']
There are 2 options for the International Plan feature
Voice Mail Plan has the following different values: ['yes' 'no']
There are 2 options for the Voice Mail Plan feature


Looking at the above columns, it is clear that while we have been give a phone number, since these are all unique, this isn't going to have any influence over whether a customer has churned or not, therefore the next steps will be to drop this column.

After this, looking at our brief exploratory data above there are a few different steps to complete before the data is ready to be passed into a model. These are:
- One hot encode all of the categorical columns
- Normalize all of the numerical columns
- Account for the class imbalance within the target data by using SMOTE

Once these steps have been completed the data should be ready to pass into our proposed models

In [37]:
X = df.drop(columns=['churn', 'phone number'], axis=1)

y = df.churn

In [38]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, stratify=y, random_state=42)

In [39]:
y_train.value_counts()

0    1995
1     338
Name: churn, dtype: int64

## Pre-Processing of the data
In this section we will work on:
- One hot encoding the relevant columns (only for the training dataset)
- Standardise the numerical columns

In [40]:
def additional_churn_features(X):
    
    X['total_calls'] = X['total day calls'] + X['total eve calls'] + X['total night calls'] + X['total intl calls']
    X['total_charges'] = X['total day charge'] + X['total eve charge'] + X['total night charge'] + X['total intl charge']
    X['total_minutes'] = X['total day minutes'] + X['total eve minutes'] + X['total night minutes'] + X['total intl minutes']
    X['pct_intl_calls'] = X['total intl calls'] / X['total_calls']
    X['pct_domestic_calls'] = 1 - X['pct_intl_calls']
    X['pct_intl_minutes'] = X['total intl minutes'] / X['total_minutes']
    X['pct_intl_charges'] = X['total intl charge'] / X['total_charges']
    X['avg_mins_per_call_day'] = X['total day minutes'] / X['total day calls']
    X['avg_mins_per_call_eve'] = X['total eve minutes'] / X['total eve calls']
    X['avg_mins_per_call_night'] = X['total night minutes'] / X['total night calls']
    
    X.fillna(method='bfill', inplace=True)
    
    
    #X['avg_mins_per_call_day'] = X['avg_mins_per_call_day'].astype('float')
    #X['avg_mins_per_call_eve'] = X['avg_mins_per_call_eve'].astype('float')
    
    return X

In [47]:
def preprocessing_pipeline(X, model=DecisionTreeClassifier):
      
      # feature engineering
    feat_eng_transformer = FunctionTransformer(additional_churn_features, validate=False)
    
    # OHE
    cat_features = X.select_dtypes(exclude=[np.number]).columns.tolist()
    cat_col_transformer = ColumnTransformer(transformers=[
    ("ohe", OneHotEncoder(handle_unknown='ignore', sparse=True), cat_features)], remainder="passthrough")
    
    
     # SMOTE & StandardScaler
    imb_pipe = ImPipeline(steps=[('feat_eng', feat_eng_transformer),
                                ('cat_ohe', cat_col_transformer),
                                ('sscaler', StandardScaler(with_mean=False)),
                                ('smote', SMOTE(random_state=42)),
                                ('model', model(random_state=42))
                                ])
    
    

  #  transformed_data = imb_pipe.fit_transform(X)
    
  #  encoder = cat_col_transformer.named_transformers_["ohe"]
  #  category_labels = encoder.categories_[0]
  #  all_cols = list(category_labels) + num_col_names
   
    return imb_pipe #imb_pipe

In [48]:
result_dt = preprocessing_pipeline(X_train)

result_dt

Pipeline(steps=[('feat_eng',
                 FunctionTransformer(func=<function additional_churn_features at 0x7fae3f692f70>)),
                ('cat_ohe',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['state', 'area code',
                                                   'international plan',
                                                   'voice mail plan'])])),
                ('sscaler', StandardScaler(with_mean=False)),
                ('smote', SMOTE(random_state=42)),
                ('model', DecisionTreeClassifier(random_state=42))])

In [49]:
smote_dt_model = result_dt.fit(X_train, y_train)

In [50]:
smote_model_pred = smote_dt_model.score(X_test, y_test)

smote_model_pred

0.91

In [51]:
y_pred = smote_dt_model.predict(X_test)

print(classification_report(y_test, y_pred, digits=3))

              precision    recall  f1-score   support

           0      0.968     0.925     0.946       855
           1      0.650     0.821     0.726       145

    accuracy                          0.910      1000
   macro avg      0.809     0.873     0.836      1000
weighted avg      0.922     0.910     0.914      1000



## Set up baseline for all models

In [58]:
model_list = [DecisionTreeClassifier, RandomForestClassifier, LogisticRegression, 
               AdaBoostClassifier, GradientBoostingClassifier, XGBClassifier]

for model in model_list:
    
    #model_performance['model_name'] = f'{model}'
    
    result = preprocessing_pipeline(X_train, model)
    
    fitted_model = result.fit(X_train, y_train)
    
    recall_train = recall_score(y_train, fitted_model.predict(X_train))
    recall_test = recall_score(y_test, fitted_model.predict(X_test))
    #model_performance['recall_train'] = recall_train
    #model_performance['recall_test'] = recall_test

    f1_train = f1_score(y_train, fitted_model.predict(X_train))
    f1_test = f1_score(y_test, fitted_model.predict(X_test))
    #model_performance['f1_train'] = f1_train
    #model_performance['f1_test'] = f1_test
    
    print(f'This is a {model} model')
    print(classification_report(y_test, fitted_model.predict(X_test), digits=3))
    

This is a <class 'sklearn.tree._classes.DecisionTreeClassifier'> model
              precision    recall  f1-score   support

           0      0.968     0.925     0.946       855
           1      0.650     0.821     0.726       145

    accuracy                          0.910      1000
   macro avg      0.809     0.873     0.836      1000
weighted avg      0.922     0.910     0.914      1000

This is a <class 'sklearn.ensemble._forest.RandomForestClassifier'> model
              precision    recall  f1-score   support

           0      0.958     0.977     0.967       855
           1      0.844     0.745     0.791       145

    accuracy                          0.943      1000
   macro avg      0.901     0.861     0.879      1000
weighted avg      0.941     0.943     0.942      1000

This is a <class 'sklearn.linear_model._logistic.LogisticRegression'> model
              precision    recall  f1-score   support

           0      0.931     0.779     0.848       855
           1    

## Section to be removed (Code used to preprocess without a pipeline)

In [53]:
X_train_cat = X_train.select_dtypes(include='object')

ohe = OneHotEncoder(handle_unknown='ignore', sparse=False)

X_train_cat_encoded = ohe.fit_transform(X_train_cat)
X_train_cat_encoded.shape

(2333, 58)

In [46]:
num_col_names = ['account length',
                 'number vmail messages',
                 'total day minutes',
                 'total day calls',
                 'total day charge',
                 'total eve minutes',
                 'total eve calls',
                 'total eve charge',
                 'total night minutes',
                 'total night calls',
                 'total night charge',
                 'total intl minutes',
                 'total intl calls',
                 'total intl charge',
                 'customer service calls',
                 'total_calls',
                 'total_charges',
                 'total_minutes',
                 'pct_intl_calls',
                 'pct_domestic_calls',
                 'pct_intl_minutes',
                 'pct_intl_charges',
                 'avg_mins_per_call_day',
                 'avg_mins_per_call_eve',
                 'avg_mins_per_call_night']

In [ ]:
X_train_cat_encoded = pd.DataFrame(
    X_train_cat_encoded,
    columns=ohe.get_feature_names(),
    index=X_train.index
)

In [ ]:
X_train.drop(columns=X_train_cat.columns.values, axis=1, inplace=True)
X_train = pd.concat([X_train_cat_encoded, X_train], axis=1)

X_train

In [ ]:
X_train.info()

## Scaling numerical features

In [ ]:
scaler = StandardScaler()

scaled_data = scaler.fit_transform(X_train_cat_encoded)
scaled_data
#scaler.transform(X_test)